In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
preprocessed_images = []
feature_vectors = []

In [3]:
def extract_hog_features(image, cell_size, num_orientations):

  # Calculate the HOG features
  hog = cv2.HOGDescriptor(_winSize=(image.shape[1] // cell_size[1] * cell_size[1],
                                      image.shape[0] // cell_size[0] * cell_size[0]),
                          _blockSize=(cell_size[1] * 2, cell_size[0] * 2),
                          _blockStride=(cell_size[1], cell_size[0]),
                          _cellSize=(cell_size[1], cell_size[0]),
                          _nbins=num_orientations)
  features = hog.compute(image)

  # Return the features
  return features

In [4]:
def hog_preprocessing(image):
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    filtered_image  = cv2.GaussianBlur(grayscale_image, (3, 3), 1.0)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    normalized_image = clahe.apply(filtered_image)
    img_array = np.array(normalized_image, dtype=np.float32) / 255.0
    corrected_array = np.power(img_array, 0.5)
    corrected_image = np.uint8(corrected_array * 255.0)
    resized_image = cv2.resize(corrected_image,(1280,1280))
    return resized_image

In [5]:
def load_images_from_folder(gender, number):

    # Set the path to the folder containing the images
    folder_path = '../data/' + gender + '/' + str(number)

    # Create an empty list to store the images
    feature_vectors = []

    # Create an empty list to store the labels
    labels = []

    # Loop through all the images in the folder
    for imagefile in os.listdir(folder_path):
        # Read the image 
        image = cv2.imread(os.path.join(folder_path, imagefile))

        # Convert the color channels from BGR to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        preprocessed_image = hog_preprocessing(image)

        feature_vector = extract_hog_features(preprocessed_image, np.array([8, 8]), 9)

        # Append the image to the list of images
        feature_vectors.append(feature_vector)

        # Append the label to the list of labels
        labels.append(number)

        del image

    return feature_vectors, labels

In [6]:
def load_dataset():
    X = []
    Y = []
    x, y = load_images_from_folder('men', 0)
    X += x
    Y += y
    x, y = load_images_from_folder('Women', 0)
    X += x
    Y += y
    x, y = load_images_from_folder('men', 1)
    X += x
    Y += y
    x, y = load_images_from_folder('Women', 1)
    X += x
    Y += y
    x, y = load_images_from_folder('men', 2)
    X += x
    Y += y
    x, y = load_images_from_folder('Women', 2)
    X += x
    Y += y
    x, y = load_images_from_folder('men', 3)
    X += x
    Y += y
    x, y = load_images_from_folder('Women', 3)
    X += x
    Y += y
    x, y = load_images_from_folder('men', 4)
    X += x
    Y += y
    x, y = load_images_from_folder('Women', 4)
    X += x
    Y += y
    x, y = load_images_from_folder('men', 5)
    X += x
    Y += y
    x, y = load_images_from_folder('Women', 5)
    X += x
    Y += y

    return np.array(X), np.array(Y)

In [7]:
# Load the dataset
X, y = load_dataset()

In [8]:
# split it into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Create a k-NN classifier with k=5
knn = KNeighborsClassifier(n_neighbors=5)

In [10]:
# Train the k-NN classifier on the training data
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [20]:
# Make predictions on the testing data
y_pred = []
for i in range(0,len(X_test)):
    y_pred_subset = knn.predict(X_test[i].reshape(1, -1))
    y_pred.append(y_pred_subset)

# y_pred = (knn.predict(x) for x in)

MemoryError: bad allocation

In [ ]:
# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy*100, '%')

Accuracy: 25.0 %
